In [1]:
import torch  
import torch.nn.functional as F 
import numpy as np
import math
from PIL import Image


def gaussian(window_size, sigma):
    """
    Generates a list of Tensor values drawn from a gaussian distribution with standard
    diviation = sigma and sum of all elements = 1.

    Length of list = window_size
    """    
    gauss =  torch.Tensor([math.exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()


def create_window(window_size, channel=1):

    # Generate an 1D tensor containing values sampled from a gaussian distribution
    _1d_window = gaussian(window_size=window_size, sigma=1.5).unsqueeze(1)
    
    # Converting to 2D  
    _2d_window = _1d_window.mm(_1d_window.t()).float().unsqueeze(0).unsqueeze(0)
     
    window = torch.Tensor(_2d_window.expand(channel, 1, window_size, window_size).contiguous())

    return window



def ssim(img1, img2, val_range, window_size=11, window=None, size_average=True, full=False):

    L = val_range # L is the dynamic range of the pixel values (255 for 8-bit grayscale images),

    pad = window_size // 2
    
    try:
        _, channels, height, width = img1.size()
    except:
        channels, height, width = img1.size()

    # if window is not provided, init one
    if window is None: 
        real_size = min(window_size, height, width) # window should be atleast 11x11 
        window = create_window(real_size, channel=channels).to(img1.device)
    
    # calculating the mu parameter (locally) for both images using a gaussian filter 
    # calculates the luminosity params
    mu1 = F.conv2d(img1, window, padding=pad, groups=channels)
    mu2 = F.conv2d(img2, window, padding=pad, groups=channels)
    
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2 
    mu12 = mu1 * mu2

    # now we calculate the sigma square parameter
    # Sigma deals with the contrast component 
    sigma1_sq = F.conv2d(img1 * img1, window, padding=pad, groups=channels) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=pad, groups=channels) - mu2_sq
    sigma12 =  F.conv2d(img1 * img2, window, padding=pad, groups=channels) - mu12

    # Some constants for stability 
    C1 = (0.01 ) ** 2  # NOTE: Removed L from here (ref PT implementation)
    C2 = (0.03 ) ** 2 

    contrast_metric = (2.0 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    contrast_metric = torch.mean(contrast_metric)

    numerator1 = 2 * mu12 + C1  
    numerator2 = 2 * sigma12 + C2
    denominator1 = mu1_sq + mu2_sq + C1 
    denominator2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (numerator1 * numerator2) / (denominator1 * denominator2)

    if size_average:
        ret = ssim_score.mean() 
    else: 
        ret = ssim_score.mean(1).mean(1).mean(1)
    
    if full:
        return ret, contrast_metric
    
    return ret


tensorify = lambda x: torch.Tensor(x.transpose((2, 0, 1))).unsqueeze(0).float().div(255.0)

load_images = lambda x: np.asarray(Image.open(x).resize((480, 640)))

img1 = load_images("./obamatest6.jpg")

img2 = load_images("./zucktest3.jpeg")

_img1 = tensorify(img1)
_img2 = tensorify(img2)
true_vs_false = ssim(_img1, _img2, val_range=255)
print("True vs False Image SSIM Score:", true_vs_false)


/var/folders/k3/5pd3dh6x5l14qp9rwq3glw0w0000gn/T/ipykernel_23939/1168220827.py:89: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025536809/work/torch/csrc/utils/tensor_numpy.cpp:212.)
  tensorify = lambda x: torch.Tensor(x.transpose((2, 0, 1))).unsqueeze(0).float().div(255.0)


True vs False Image SSIM Score: tensor(0.3385)


In [25]:
import torch

from torchmetrics.image.kid import KernelInceptionDistance
kid = KernelInceptionDistance(subset_size=4)

def load_and_convert_images(file_path):
    img = Image.open(file_path).resize((480, 640))
    img_array = np.array(img)
    img_tensor = torch.tensor(img_array, dtype=torch.uint8)
    img_tensor = img_tensor.permute(2, 0, 1)  # Ensure correct dimension order: channels, height, width
    return img_tensor.unsqueeze(0)  # Add batch dimension

# Define your images paths
img1_path = "./obamatest6.jpg"
img2_path = "./zucktest3.jpeg"

# Load and convert images
img1 = load_and_convert_images(img1_path)
img2 = load_and_convert_images(img2_path)

kid.update(img1, real=True)
kid.update(img1, real=False)
kid.update(img1, real=True)
kid.update(img1, real=False)
kid.update(img1, real=True)
kid.update(img1, real=False)
kid.update(img1, real=True)
kid.update(img1, real=False)
kid.update(img1, real=True)
kid.update(img1, real=False)

kid_mean, kid_std = kid.compute()
print((kid_mean, kid_std))

tensor([[[[ 0,  0,  0,  ...,  4,  4,  4],
          [ 0,  0,  0,  ...,  4,  4,  4],
          [ 0,  0,  0,  ...,  4,  4,  4],
          ...,
          [ 3,  3,  3,  ...,  2,  2,  2],
          [ 2,  2,  2,  ...,  2,  2,  2],
          [ 2,  2,  2,  ...,  2,  2,  2]],

         [[13, 13, 13,  ...,  0,  0,  0],
          [13, 13, 13,  ...,  0,  0,  0],
          [13, 13, 13,  ...,  0,  0,  0],
          ...,
          [ 5,  5,  5,  ...,  1,  1,  1],
          [ 4,  4,  4,  ...,  1,  1,  1],
          [ 4,  4,  4,  ...,  1,  1,  1]],

         [[22, 22, 22,  ..., 17, 17, 17],
          [22, 22, 22,  ..., 17, 17, 17],
          [22, 22, 22,  ..., 17, 17, 17],
          ...,
          [18, 18, 18,  ...,  6,  6,  6],
          [16, 16, 16,  ...,  6,  6,  6],
          [16, 16, 16,  ...,  6,  6,  6]]]], dtype=torch.uint8)
tensor([[[[ 0,  0,  0,  ..., 12, 12, 12],
          [ 0,  0,  0,  ..., 12, 12, 12],
          [ 0,  0,  0,  ..., 12, 12, 12],
          ...,
          [14, 14, 14,  ...,  8,